In [ ]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None 
data = pd.read_excel('stock kos.xlsx')
stock = data[['Date','Product ID','Product Name','Start Available Stock','Inbound from PO','Inbound from SO Act Qty']]
stock['Total Qty Day'] = stock['Inbound from PO']+stock['Start Available Stock']

In [ ]:
import plotly.express as px
from ipywidgets import interact


# Ensure date column is in datetime format
stock['Date'] = pd.to_datetime(stock['Date'])

# Get the start of the week (Monday)
stock['Week Start'] = stock['Date'] - pd.to_timedelta(stock['Date'].dt.weekday, unit='D')

# Extract month in YYYY-MM format
stock['Month'] = stock['Date'].dt.strftime('%Y-%m')

# Aggregate max total stock per week and per month
weekly_max = stock.groupby(['Product ID', 'Week Start'])['Total Qty Day'].max().reset_index()
monthly_max = stock.groupby(['Product ID', 'Month'])['Total Qty Day'].max().reset_index()

# Create a dictionary mapping Product ID to Product Name
product_mapping = stock[['Product ID', 'Product Name']].drop_duplicates().set_index('Product ID')['Product Name'].to_dict()

# Interactive plot function
def plot_stock(product_id, timeframe):
    product_name = product_mapping.get(product_id, "Unknown Product")  # Get product name
    
    if timeframe == 'Weekly':
        data = weekly_max[weekly_max['Product ID'] == product_id]
        x_col = 'Week Start'
    else:
        data = monthly_max[monthly_max['Product ID'] == product_id]
        x_col = 'Month'
    
    # Display product name
    print(f"Selected Product: {product_name} (ID: {product_id})")
    
    # Plot the graph
    fig = px.line(data, x=x_col, y='Total Qty Day', 
                  title=f'Max Total Qty for {product_name} ({product_id})', markers=True)
    fig.show()

# Dropdown filter using interact
interact(plot_stock, product_id=stock['Product ID'].unique(), timeframe=['Weekly', 'Monthly'])